#### Lecture 1. 정점 표현 방법

1. 정점 표현 학습
- 정점 임베딩(Node Embedding) : 그래프의 정점들을 벡터의 형태로 표현하는 것
- 임베딩 공간 : 정점이 표현되는 벡터 공간, 입력은 그래프, 출력은 벡터 표현
- 벡터 형태 데이터 도구를 그래프에 적용 가능(분류기, 군집 분석 알고리즘 등)
- 그래프에서 정점간 유사도를 임베딩 공간에서도 보존 (벡터 유사도는 내적을 사용)

2. 인접성(adjacency) 기반 접근법
- 두 정점이 인접할 때 유사, 인접하다는 것은 둘을 연결하는 간선이 있음을 의미
- 손실함수 $\mathcal{L} = \sum_{(u,v) \in V\times V} \lVert z_u^T z_v - A_{u,v} \rVert^2$ 를 최소화
- 그래프 상의 거리, 군집 관계를 무시하는 한계

3. 다른 접근법
- 거리 기반 접근법 : 두 정점 사이의 거리가 일정 기준 이내일 때 유사하다고 간주
- 경로 기반 접근법 : 두 정점 사이의 경로가 많을수록 유사
- 중첩 기반 접근법 : 두 정점이 많은 이웃을 공유할수록 유사
    - 공통이웃수 $S_{u,v} = \lvert N(u) \cap N(v) \rvert = \sum_{w \in N(u) \cap N(v)} 1$
    - 자카드 유사도 : 공통이웃의 수대신 비율 계산 $ \lvert N(u) \cap N(v) \rvert / \lvert N(u) \cup N(v) \rvert $
    - Adamic Adar : 공통이웃 각각에 가중치를 부여하여 가중합을 계산 $\sum_{w \in N(u) \cap N(v)} \frac{1}{d_w}$
        - 연결성이 높은 이웃을 함께 이웃으로 둔다는 사실이 둘을 유사하다고 할수는 없음

4. 임의보행 기반 접근법
- 한 정점에서 시작하여 이웃중 하나를 균일한 확률로 선택하여 이동(임의보행) 할 때 다른 정점에 도달할 확률을 유사도로 간주
- 손실함수 $\mathcal{L} = \sum_{u \in V}\sum_{v \in N_R(u)} -log(P(v|z_u))$
- $P(v|z_u)=\frac{exp(z_u^T z_v)}{\sum_{n \in V}exp(z_u^Tz_n)}$ : u에서 시작한 임의보행이 v에 도달할 확률을 임베딩으로부터 추정한 결과
- DeepWalk : 임의 보행 시 이동 확률이 균일
- Node2Vec : 2차 치우친 임의보행 사용 , 직전 정점의 거리를 기준으로 차등적인 확률 부여
    - 멀어지는 방향에 높은 확률 : 정점의 역할(다리역할, 연결성)에 따라 임베딩이 유사
    - 가까워지는 방향에 높은 확률 : 같은 군집에 속하면 임베딩이 유사
- 손실함수 계산시 정점 수의 제곱에 비례하는 시간이 소요됨
    - 근사식 사용 : 몇 개의 정점만 뽑아서 비교(네거티브 샘플)

5. 변환식 정점 표현 학습과 귀납식 정점 표현 학습
- 변환식(transdctive) : 정점의 임베딩 자체를 얻음  ENC(v) = Zu
- 귀납식(inductive) : 정점을 임베딩으로 변화시키는 함수를 얻음, 그래프 신경망
- 변환식의 한계
    - 학습 진행 이후 추가된 정점에 대해서 임베딩을 얻을 수 없음
    - 모든 정점에 대해 미리 계산 후 저장
    - 정점의 속성 정보 활용 불가

#### Lecture 2. Graph와 추천 시스템

1. 넷플릭스 챌린지 개요
- 훈련 데이터 48만명 사용자의 1만 8천개 영화에 대한 1억개의 평점 데이터 / 평가 데이터 최신평점 280만개
- 추천시스템의 성능을 10%이상 향상시키는 것이 목표(RMSE 0.9514 => 0.8563)

2. 잠재인수 모형(latent factor model, UV decomposition , SVD)
- 사용자와 영화를 벡터로 임베딩
- 고정된 인수 대신 효과적인 인수를 학습하는 것이 목표
- 사용자와 상품의 임베딩의 내적이 평점과 최대한 유사하도록 하는 것
- 평점행렬 R, 사용자 행렬 P, 상품 행렬 Q   $ R \sim Q P^T $
- 손실함수 $ \sum_{(i,x) \in R} (r_{xi} - p_x^Tq_i)^2 $
- 과적합 방지를 위해 정규화 항을 더해준 $ \sum_{(i,x) \in R} (r_{xi} - p_x^Tq_i)^2 + \lambda_1 \sum_x \lVert p_x \rVert ^2 + \lambda_2 \sum_i \lVert q_i \rVert ^2 $ 를 최소화
- 정규화는 극단적인(절대값이 큰) 임베딩을 방지하는 효과 ($\lambda_1, \lambda_2$ 는 하이퍼 파라미터 값)
- 최적화를 위해 경사하강법을 사용, 확률적 경사하강법을 사용하면 불안정하지만 학습 속도가 빠름

3. 사용자와 상품의 편향을 고려한 잠재 인수 모형
- 사용자의 편향 : 해당 사용자의 평점 평균과 전체 평점 평균의 차
- 상품의 편향 : 해당 상품에 대한 평점 평균과 전체 평점 평균의 차
- 평점 = 전체평균 + 사용자 편향 + 상품 편향 + 상호작용 으로 파악
- 손실함수 $ \sum_{(i,x) \in R} (r_{xi} - (\mu + b_x + b_i + p_x^Tq_i))^2 + \lambda_1 \sum_x \lVert p_x \rVert ^2 + \lambda_2 \sum_i \lVert q_i \rVert ^2 + \lambda_3 \sum_x b_x^2 + \lambda_4 \sum_i b_i^2 $
- 손실함수를 최소화하는 잠재 인수와 편향을 학습

4. 시간적 편향을 고려한 잠재 인수 모형
- 플랫폼 시스템의 변화로 평균 평점이 변동
- 영화의 평점은 시간이 지남에 따라 상승하는 경향이 있음

5. 넷플릭스 챌린지 결과
- BellKor 팀 앙상블 학습을 통해 최종 우승